In [17]:
!pip install huggingface_hub[hf_xet]
#!pip install hf_xet

In [5]:
!pip install ipywidgets


In [32]:
from transformers import AutoProcessor, AutoModelForCTC
import torch
import torch.nn as nn
import torch.optim as optim

In [34]:
dummy_mel_spectrogram = torch.randn(4, 128, 500)


In [36]:
class SimpleASRModel(nn.Module):
    def __init__(self, num_mels, num_classes):
        super().__init__()
        # 인코더: 멜 스펙트로그램에서 특징 추출
        # 여기서는 간단한 CNN 레이어만 예시로 사용. 실제는 더 복잡한 Conformer/Transformer 블록
        self.encoder = nn.Sequential(
            nn.Conv1d(num_mels, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv1d(256, 512, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            # 필요한 경우 추가 레이어 (RNN, Transformer Encoder 등)
            # 최종적으로 시퀀스 길이가 줄어들고 채널 수는 늘어난 특징 벡터 생성
        )
        # 디코더: 특징 벡터에서 텍스트 시퀀스 생성 (간략화)
        # 실제로는 트랜스포머 디코더 또는 LSTM/GRU 디코더 사용
        # 여기서는 마지막 특징 벡터를 클래스(토큰) 개수로 매핑하는 간단한 선형 레이어
        self.decoder_output = nn.Linear(512, num_classes) # num_classes는 토큰(문자/음소) 개수

    def forward(self, x):
        # x는 (batch_size, n_mels, time_steps) 형태의 멜 스펙트로그램
        x = self.encoder(x) # 인코딩된 특징 벡터 (batch_size, feature_dim, new_time_steps)
        # 디코딩을 위해 차원 조절 (예시)
        x = x.permute(0, 2, 1) # (batch_size, new_time_steps, feature_dim)
        output = self.decoder_output(x) # (batch_size, new_time_steps, num_classes)
        return output


In [38]:
input_dim = 128
output_dim = 100 # 예시로 100개의 가능한 출력 토큰이 있다고 가정
model = SimpleASRModel(input_dim, output_dim)


In [40]:
output = model(dummy_mel_spectrogram)
print(f"모델 출력 형태: {output.shape}")

모델 출력 형태: torch.Size([4, 125, 100])


In [42]:
import torch
import torch.nn as nn
# from torch.nn import TransformerEncoder, TransformerDecoder, TransformerEncoderLayer, TransformerDecoderLayer

# 편의상 아래처럼 임포트하는 경우가 많습니다.
from torch.nn import (
    TransformerEncoder,
    TransformerDecoder,
    TransformerEncoderLayer,
    TransformerDecoderLayer
)

# --- 멜 스펙트로그램 특징 (가상의 입력) ---
# 배치 크기=4, 멜 밴드=128, 시간 스텝=500
# Transformer는 일반적으로 (시퀀스 길이, 배치 크기, 특징 차원) 또는
# (배치 크기, 시퀀스 길이, 특징 차원) 형태를 기대합니다.
# 멜 스펙트로그램은 (배치 크기, 멜 밴드 수, 시간 스텝 수) 형태이므로,
# Transformer 입력에 맞게 차원 순서를 변경하고 (transpose)
# 멜 밴드 수를 특징 차원으로 사용하거나, 별도의 선형 변환을 거칩니다.

batch_size = 4
n_mels = 128
time_steps = 500
vocab_size = 100 # 출력 토큰(문자/음소) 개수

dummy_mel_spectrogram = torch.randn(batch_size, n_mels, time_steps)

# --- 실제 ASR 모델 (Transformer 기반, Simplified) ---
class TransformerASRModel(nn.Module):
    def __init__(self,
                 n_mels=128,          # 멜 스펙트로그램의 멜 밴드 개수 (입력 특징 차원)
                 d_model=512,         # Transformer 내부 모델 차원
                 nhead=8,             # 멀티 헤드 어텐션의 헤드 개수
                 num_encoder_layers=6, # 인코더 레이어 개수
                 num_decoder_layers=6, # 디코더 레이어 개수
                 dim_feedforward=2048, # 피드포워드 네트워크의 차원
                 dropout=0.1,
                 num_classes=100,     # 출력 토큰(문자/음소) 개수
                 max_len=1000):       # 최대 입력 시퀀스 길이 (위치 인코딩용)
        super().__init__()

        # 1. Input Embedding / Feature Projection for Encoder
        # 멜 스펙트로그램의 n_mels 차원을 d_model 차원으로 매핑
        self.input_projection = nn.Linear(n_mels, d_model)

        # 2. Positional Encoding for Encoder (시간 순서 정보를 주입)
        # 트랜스포머는 순서 정보를 직접 학습하지 않으므로 위치 인코딩이 필요
        self.positional_encoding = nn.Parameter(torch.randn(1, max_len, d_model))


        # 3. Transformer Encoder
        encoder_layer = TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True # (batch, seq_len, feature) 순서로 입력받도록 설정
        )
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)

        # 4. Target Embedding for Decoder (출력 텍스트 토큰 임베딩)
        self.tgt_embedding = nn.Embedding(num_classes, d_model)
        # 5. Positional Encoding for Decoder
        self.tgt_positional_encoding = nn.Parameter(torch.randn(1, max_len, d_model))

        # 6. Transformer Decoder
        decoder_layer = TransformerDecoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_decoder = TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)

        # 7. Output Layer
        self.output_linear = nn.Linear(d_model, num_classes)

    def forward(self, src, tgt):
        """
        Args:
            src (Tensor): 인코더 입력 (멜 스펙트로그램).
                          Shape: (batch_size, n_mels, time_steps)
            tgt (Tensor): 디코더 입력 (타겟 텍스트 토큰 ID 시퀀스).
                          Shape: (batch_size, target_seq_len)
        """
        # --- Encoder Input Processing ---
        # 멜 스펙트로그램 (batch, n_mels, time_steps) -> (batch, time_steps, n_mels)
        src = src.permute(0, 2, 1)
        # n_mels 차원을 d_model 차원으로 투영
        src = self.input_projection(src) # (batch_size, time_steps, d_model)
        # 위치 인코딩 추가
        src = src + self.positional_encoding[:, :src.size(1), :] # (batch_size, time_steps, d_model)

        # --- Encoder Forward ---
        memory = self.transformer_encoder(src) # (batch_size, time_steps, d_model)

        # --- Decoder Input Processing ---
        # 타겟 토큰 ID를 임베딩
        tgt = self.tgt_embedding(tgt) # (batch_size, target_seq_len, d_model)
        # 위치 인코딩 추가
        tgt = tgt + self.tgt_positional_encoding[:, :tgt.size(1), :] # (batch_size, target_seq_len, d_model)


        # --- Decoder Forward ---
        # 디코더 마스크 생성 (미래 토큰을 참조하지 않도록)
        # tgt_mask = nn.Transformer.generate_square_subsequent_mask(tgt.size(1)).to(tgt.device)
        # 소스 패딩 마스크 (패딩 토큰을 어텐션에서 제외)
        # src_padding_mask = (src_lengths == 0) # 실제로는 패딩 인덱스를 기반으로 마스크 생성

        # 간략화를 위해 마스크는 제외 (실제 구현에서는 필수)
        output = self.transformer_decoder(tgt, memory) # (batch_size, target_seq_len, d_model)

        # --- Output Layer ---
        output = self.output_linear(output) # (batch_size, target_seq_len, num_classes)

        return output

# --- 모델 초기화 및 더미 데이터 통과 예시 ---
# 모델 인스턴스 생성
model = TransformerASRModel(
    n_mels=n_mels,
    d_model=512, # 트랜스포머 내부 차원
    nhead=8,
    num_encoder_layers=6,
    num_decoder_layers=6,
    num_classes=vocab_size
)

# 더미 타겟 텍스트 시퀀스 (예: [START_TOKEN_ID, 10, 20, 30, END_TOKEN_ID])
# (batch_size, target_seq_len)
dummy_target_text_tokens = torch.randint(0, vocab_size, (batch_size, 30))

# 모델에 입력 (멜 스펙트로그램과 타겟 토큰 시퀀스)
output = model(dummy_mel_spectrogram, dummy_target_text_tokens)
print(f"Transformer ASR 모델 출력 형태: {output.shape}")

Transformer ASR 모델 출력 형태: torch.Size([4, 30, 100])


---
모델의 출력 형태를 계산하는 것은 딥러닝 모델을 설계하고 디버깅하는 데 있어 굉장히 중요해요. 특히 복잡한 모델이나 여러 레이어를 거칠 때는 각 레이어가 입력 형태를 어떻게 바꾸는지 이해해야 해요.

모델의 출력 형태는 주로 **입력 형태**와 모델을 구성하는 각 **레이어(층)의 특성 및 순서**에 따라 결정돼요.

### 기본 원리: 각 레이어의 변환 규칙 이해하기

각 레이어는 정해진 규칙에 따라 입력 텐서(tensor)의 형태(shape)를 변경해요. 가장 흔히 사용되는 레이어들의 기본적인 변환 규칙을 알면 모델 전체의 출력 형태를 추적할 수 있어요.

**주요 레이어별 출력 형태 계산:**

1.  **`nn.Linear` (선형 변환 / Fully Connected Layer):**
    * **입력 형태:** `(..., input_features)` (가장 마지막 차원이 입력 특징의 수)
    * **출력 형태:** `(..., output_features)` (마지막 차원이 출력 특징의 수로 바뀜)
    * **예시:** `nn.Linear(in_features=128, out_features=256)`
        * 입력이 `(batch_size, 10, 128)`이면, 출력은 `(batch_size, 10, 256)`
        * 입력이 `(batch_size, 128)`이면, 출력은 `(batch_size, 256)`

2.  **`nn.Conv1d` (1D 컨볼루션 레이어):**
    * 음성 신호 처리(예: 스펙트로그램의 시간 축)에 자주 사용돼요.
    * **입력 형태:** `(batch_size, in_channels, length)`
    * **출력 형태:** `(batch_size, out_channels, new_length)`
    * **`new_length` 계산 공식:**
        $\text{new\_length} = \lfloor \frac{\text{length} + 2 \times \text{padding} - \text{dilation} \times (\text{kernel\_size} - 1) - 1}{\text{stride}} + 1 \rfloor$
    * **예시:** `nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=2, padding=1)`
        * 입력이 `(4, 128, 500)`이면 (batch\_size, 멜 밴드 수, 시간 스텝)
        * `out_channels`는 `256`이 됩니다.
        * `new_length`는 $\lfloor \frac{500 + 2 \times 1 - 1 \times (3 - 1) - 1}{2} + 1 \rfloor = \lfloor \frac{500 + 2 - 2 - 1}{2} + 1 \rfloor = \lfloor \frac{499}{2} + 1 \rfloor = \lfloor 249.5 + 1 \rfloor = 249 + 1 = 250$
        * 출력은 `(4, 256, 250)`

3.  **`nn.Conv2d` (2D 컨볼루션 레이어):**
    * 이미지 처리나 멜 스펙트로그램 같은 2D 데이터에 사용돼요.
    * **입력 형태:** `(batch_size, in_channels, height, width)`
    * **출력 형태:** `(batch_size, out_channels, new_height, new_width)`
    * **`new_height` 계산 공식:**
        $\text{new\_height} = \lfloor \frac{\text{height} + 2 \times \text{padding}[0] - \text{dilation}[0] \times (\text{kernel\_size}[0] - 1) - 1}{\text{stride}[0]} + 1 \rfloor$
    * **`new_width` 계산 공식:**
        $\text{new\_width} = \lfloor \frac{\text{width} + 2 \times \text{padding}[1] - \text{dilation}[1] \times (\text{kernel\_size}[1] - 1) - 1}{\text{stride}[1]} + 1 \rfloor$
    * `kernel_size`, `stride`, `padding`, `dilation`이 단일 숫자면 높이/너비에 동일하게 적용되고, 튜플이면 각각 적용돼요.

4.  **`nn.MaxPool1d` / `nn.AvgPool1d` (풀링 레이어):**
    * 컨볼루션과 유사하게 길이를 줄여요.
    * **입력 형태:** `(batch_size, in_channels, length)`
    * **출력 형태:** `(batch_size, in_channels, new_length)`
    * **`new_length` 계산 공식:** 컨볼루션과 동일한 공식에 `kernel_size`, `stride`, `padding`, `dilation` 값을 대입해요.

5.  **`nn.RNN`, `nn.LSTM`, `nn.GRU` (RNN 계열):**
    * **입력 형태:** `(seq_len, batch_size, input_size)` 또는 `(batch_size, seq_len, input_size)` ( `batch_first=True`일 경우)
    * **출력 형태:** `(seq_len, batch_size, hidden_size * num_directions)` 또는 `(batch_size, seq_len, hidden_size * num_directions)`
    * `hidden_size`는 RNN 셀의 은닉 상태 크기, `num_directions`는 양방향(bidirectional)일 경우 2, 단방향일 경우 1이에요.

6.  **`nn.TransformerEncoderLayer`, `nn.TransformerDecoderLayer`:**
    * 내부적으로 복잡한 어텐션과 피드포워드 네트워크를 거치지만, **기본적으로 입력 시퀀스 길이와 특징 차원(`d_model`)은 유지**돼요.
    * **입력 형태:** `(seq_len, batch_size, d_model)` 또는 `(batch_size, seq_len, d_model)`
    * **출력 형태:** 입력 형태와 **동일**해요. (Multi-Head Attention 등의 내부 연산 후에도 최종 출력 형태는 유지)

7.  **`nn.Flatten`:**
    * **입력 형태:** `(batch_size, C, H, W)` 또는 `(batch_size, H, W)` 등
    * **출력 형태:** `(batch_size, C * H * W)` 또는 `(batch_size, H * W)` 등 (배치 차원을 제외한 나머지 차원들을 일렬로 펼침)

### 출력 형태 계산 방법:

1.  **손으로 직접 계산:**
    각 레이어의 공식과 현재 텐서의 형태를 바탕으로 순서대로 계산해나가요. 이는 모델의 개념을 이해하는 데 가장 좋은 방법이에요.

2.  **더미 데이터(Dummy Data)를 이용한 확인:**
    가장 빠르고 정확한 방법이에요. 실제 모델에 들어갈 것과 동일한 형태의 임의의 텐서(더미 데이터)를 만들어서 모델의 `forward` 메서드를 통과시켜보면 돼요.

    ```python
    import torch
    import torch.nn as nn

    # 예시 모델
    class MyModel(nn.Module):
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=2, padding=1)
            self.relu1 = nn.ReLU()
            self.conv2 = nn.Conv1d(in_channels=256, out_channels=512, kernel_size=3, stride=2, padding=1)
            self.relu2 = nn.ReLU()
            # conv2 이후의 출력 형태를 알 수 없다고 가정
            # self.flatten = nn.Flatten() # 만약 Flatten을 쓴다면
            # self.linear = nn.Linear(?, 10) # ?를 찾아야 함

        def forward(self, x):
            x = self.relu1(self.conv1(x))
            x = self.relu2(self.conv2(x))
            print(f"After conv2: {x.shape}") # 중간 출력 형태 확인
            # x = self.flatten(x)
            # x = self.linear(x)
            return x

    # 더미 입력 생성 (예: batch_size=4, n_mels=128, time_steps=500)
    dummy_input = torch.randn(4, 128, 500)

    model = MyModel()
    output = model(dummy_input)
    print(f"Final output shape: {output.shape}")
    ```

3.  **라이브러리 활용 (`torchsummary`, `torchinfo`):**
    모델의 요약 정보를 출력해주는 유용한 라이브러리들이 있어요. 이들은 각 레이어별 입출력 형태와 파라미터 수를 자동으로 계산해서 보여줘요.

    * **`torchsummary`:** (설치: `pip install torchsummary`)
        ```python
        from torchsummary import summary

        # 위 MyModel 정의 후
        model = MyModel()
        summary(model, input_size=(128, 500)) # (in_channels, length)
        ```
    * **`torchinfo`:** (설치: `pip install torchinfo`)
        ```python
        from torchinfo import summary

        # 위 MyModel 정의 후
        model = MyModel()
        summary(model, input_size=(4, 128, 500)) # (batch_size, in_channels, length)
        ```

    이 라이브러리들을 사용하면 복잡한 모델의 형태를 한눈에 파악할 수 있어서 디버깅 및 설계에 큰 도움이 돼요.

모델의 출력 형태를 정확히 아는 것은 다음 레이어의 입력 형태로 연결하거나, 최종 출력 레이어의 크기를 설정하는 등 모델 설계의 필수적인 단계입니다.

In [7]:
import torch
import torch.nn as nn
from transformers  import AutoProcessor,AutoModelForCTC

In [21]:
# 1. 사전 학습된 ASR 모델 로드 (Wav2Vec2)
model_name = "facebook/wav2vec2-base-960h"
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForCTC.from_pretrained(model_name)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
new_vocab_size=30
model.lm_head=nn.Linear(model.config.hidden_size,new_vocab_size)

In [25]:
optimy= torch.optim.AdamW(model.parameters(), lr=1e-5) 

In [27]:
print("\n--- 파인튜닝 설정 완료 ---")
for name, param in model.named_parameters():
    # 학습 가능한 파라미터만 출력 (대부분 True일 것)
    if param.requires_grad:
        print(f"Layer: {name}, requires_grad: {param.requires_grad}")


--- 파인튜닝 설정 완료 ---
Layer: wav2vec2.masked_spec_embed, requires_grad: True
Layer: wav2vec2.feature_extractor.conv_layers.0.conv.weight, requires_grad: True
Layer: wav2vec2.feature_extractor.conv_layers.0.layer_norm.weight, requires_grad: True
Layer: wav2vec2.feature_extractor.conv_layers.0.layer_norm.bias, requires_grad: True
Layer: wav2vec2.feature_extractor.conv_layers.1.conv.weight, requires_grad: True
Layer: wav2vec2.feature_extractor.conv_layers.2.conv.weight, requires_grad: True
Layer: wav2vec2.feature_extractor.conv_layers.3.conv.weight, requires_grad: True
Layer: wav2vec2.feature_extractor.conv_layers.4.conv.weight, requires_grad: True
Layer: wav2vec2.feature_extractor.conv_layers.5.conv.weight, requires_grad: True
Layer: wav2vec2.feature_extractor.conv_layers.6.conv.weight, requires_grad: True
Layer: wav2vec2.feature_projection.layer_norm.weight, requires_grad: True
Layer: wav2vec2.feature_projection.layer_norm.bias, requires_grad: True
Layer: wav2vec2.feature_projection.proje

네, 코드상에서 **전이 학습**과 **파인튜닝**의 구현 차이점은 주로 **모델의 어떤 부분(레이어)을 학습시킬 것인가**에 따라 달라집니다. PyTorch에서는 이를 **각 레이어의 `requires_grad` 속성**을 조절함으로써 구현합니다.

### 1. 전이 학습 (Pre-trained Model as Feature Extractor) 방식의 코드 구현

이 방식은 사전 학습된 모델의 **대부분을 고정(Freeze)시키고**, 주로 **마지막 출력 레이어만 새로 만들어서 학습**시키는 방법입니다. 마치 사전 학습 모델이 데이터에서 고수준의 특징(feature)을 뽑아주는 역할만 하고, 이 특징을 받아서 새로운 작업을 수행하는 작은 분류기(classifier)만 학습하는 것과 같습니다.

**코드 구현의 핵심:**
* 사전 학습된 모델의 모든 `parameters()`에 대해 `requires_grad = False`로 설정하여 **기울기 계산을 비활성화**하고, 따라서 **가중치 업데이트가 일어나지 않도록** 합니다.
* 모델의 **마지막 출력 레이어**(`nn.Linear` 등)를 새로운 작업(새로운 클래스 수)에 맞게 **재정의**합니다. 이 새로 정의된 레이어는 기본적으로 `requires_grad = True`이므로 학습이 진행됩니다.
* 옵티마이저를 정의할 때, **`requires_grad = True`인 파라미터만 전달**합니다.

**예시 코드 (컴퓨터 비전 모델 ResNet18을 예시로, ASR도 유사하게 적용 가능):**

```python
import torch
import torch.nn as nn
import torchvision.models as models # 비전 모델 예시 (ASR도 유사)

# 1. 사전 학습된 모델 로드 (예: ImageNet으로 학습된 ResNet18)
# ASR에서는 Wav2Vec2, Whisper 등 사전 학습된 ASR 모델을 로드합니다.
model = models.resnet18(weights='DEFAULT') # weights=ResNet18_Weights.IMAGENET1K_V1

# 2. 모든 파라미터 고정 (Freeze)
# 이 부분이 '전이 학습'의 특징을 가장 잘 나타냅니다.
for param in model.parameters():
    param.requires_grad = False # 기울기 계산 비활성화 -> 가중치 업데이트 안 됨

# 3. 마지막 출력 레이어만 새로운 작업에 맞게 수정
# ResNet18의 마지막 fully connected layer는 'fc'라는 이름으로 접근 가능.
# ASR 모델의 경우, 마지막 ASR 헤드(예: `lm_head` 또는 `decoder_output`)를 변경합니다.
num_ftrs = model.fc.in_features # 기존 마지막 레이어의 입력 피처 수
num_classes = 10 # 새로운 분류 작업의 클래스 수 (예: 10개 음성 명령)
model.fc = nn.Linear(num_ftrs, num_classes) # 새로운 레이어는 기본적으로 requires_grad=True

# 4. 옵티마이저 정의
# 새로 추가된 (학습 가능한) 파라미터만 옵티마이저에 전달.
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001) # model.fc.parameters()만 학습

# 5. 학습 루프 (생략)
# 이 모델을 가지고 새로운 데이터셋으로 학습을 진행하면,
# model.fc 레이어만 업데이트되고, 나머지 사전 학습된 레이어의 가중치는 고정됩니다.

print("--- 전이 학습 (Feature Extractor) 설정 완료 ---")
for name, param in model.named_parameters():
    print(f"Layer: {name}, requires_grad: {param.requires_grad}")
```

### 2. 파인튜닝 (Fine-tuning) 방식의 코드 구현

파인튜닝은 사전 학습된 모델의 **모든 또는 대부분의 레이어를 새로운 데이터셋에 맞춰 미세 조정**하는 방식입니다. 초기 가중치로 사전 학습된 가중치를 사용하되, 이를 새로운 데이터에 최적화하도록 학습합니다.

**코드 구현의 핵심:**
* 사전 학습된 모델을 로드한 후, **모든 파라미터 또는 대부분의 파라미터에 대해 `requires_grad = True` 상태를 유지**합니다. (기본값이 `True`이므로 특별히 건드리지 않아도 됩니다.)
* 필요하다면 (클래스 개수가 다르다면) 마지막 출력 레이어만 수정합니다.
* **학습률(Learning Rate)을 매우 작게 설정**합니다 (예: $10^{-5}$ 또는 $10^{-6}$). 이는 사전 학습된 지식을 급격히 훼손하는 것을 방지하고, 미세한 조정이 이루어지도록 합니다.
* 때로는 **차등 학습률(Discriminative Learning Rates)**을 사용하여, 초기 레이어는 더 작은 학습률로, 후반부 레이어는 조금 더 큰 학습률로 학습하기도 합니다.

**예시 코드 (ASR 모델 Wav2Vec2를 예시로):**

```python
import torch
import torch.nn as nn
from transformers import AutoProcessor, AutoModelForCTC # Hugging Face Transformers

# 1. 사전 학습된 ASR 모델 로드 (Wav2Vec2)
model_name = "facebook/wav2vec2-base-960h"
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForCTC.from_pretrained(model_name)

# 2. 마지막 출력 레이어 수정 (필요하다면)
# Wav2Vec2의 경우, `lm_head`가 최종 분류기 역할
new_vocab_size = 30 # 예: 한국어 음성 인식을 위한 새로운 토큰(문자) 개수
model.lm_head = nn.Linear(model.config.hidden_size, new_vocab_size)

# 3. 모든 파라미터(또는 대부분)가 학습 가능하도록 설정
# Hugging Face 모델은 기본적으로 모든 파라미터가 requires_grad=True로 로드됩니다.
# 따라서 이 부분은 별도로 코드를 작성할 필요가 없습니다.
# 만약 특정 레이어를 고정하고 싶다면 for param in model.some_layer.parameters(): param.requires_grad = False 처럼 설정합니다.

# 4. 옵티마이저 정의
# 모든 학습 가능한 파라미터를 옵티마이저에 전달합니다.
# **파인튜닝의 핵심: 낮은 학습률**
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5) # 매우 낮은 학습률

# 5. 학습 루프 (생략)
# 이 모델을 가지고 새로운 데이터셋으로 학습을 진행하면,
# 모든 레이어의 가중치가 낮은 학습률로 미세 조정됩니다.

print("\n--- 파인튜닝 설정 완료 ---")
for name, param in model.named_parameters():
    # 학습 가능한 파라미터만 출력 (대부분 True일 것)
    if param.requires_grad:
        print(f"Layer: {name}, requires_grad: {param.requires_grad}")

```

### 핵심 차이점 정리 (코드 관점):

* **`param.requires_grad = False`**: 특정 레이어의 가중치를 **고정(Freeze)**시켜 학습되지 않게 합니다. (전이 학습 - 특징 추출기 방식에서 주로 사용)
* **옵티마이저에 전달하는 파라미터**: `optimizer = torch.optim.Adam(model.parameters(), ...)`에서 `model.parameters()` 대신 `model.fc.parameters()`와 같이 **일부만 전달**하면 그 부분만 학습됩니다. (`requires_grad=False`인 파라미터는 어차피 기울기가 계산되지 않아 학습에 포함되지 않습니다.)
* **학습률(Learning Rate)**: 파인튜닝 시에는 일반적으로 **매우 낮은 학습률**을 사용합니다.

이러한 코드상의 차이점을 이해하고 활용하는 것이 전이 학습과 파인튜닝을 성공적으로 적용하는 핵심입니다.